<a href="https://colab.research.google.com/github/bbberylll/ESAA/blob/main/%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spaceship Titanic 수상작 리뷰

## 사용한 데이터 & 주제 설명
### 대회 목표
: 우주선 탑승객 생존 여부(Transported: True/False) 예측 (이진 분류)

### 데이터 구성:
- 승객 정보 및 탑승 기록 (ex. PassengerId, HomePlanet, CryoSleep, Cabin, Destination, Age, VIP, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck, Transported)

- 범주형 + 수치형 변수 혼합

- 다수 결측치 존재, 특성 공학(피처 엔지니어링) 중요

### 평가지표
: Accuracy 또는 AUC

## 1. 어떤 기법이 적용됐는지?

### 모델:
- XGBoost Classifier
- LightGBM Classifier

### 블렌딩 방식:
- soft voting 전략 (확률 예측을 가중 평균)
- 블렌딩 계수는 검증 데이터 기반 조정

### Preprocessing:
- 결측치 imputation (Age, Cabin 구분 등)
- 범주형 변수 인코딩 (Label Enc, One-Hot)
- 파생변수 생성 (Deck, NumRooms, TotalSpend 등)

## 문제 접근 방식

### 데이터 전처리
1. Cabin 문자열에서 데크 분리 (Deck, NumCabin)
2. GroupID 생성 (동일한 스택 탑승객별 그룹)
3. 지출 관련 피쳐 통합 및 로그 스케일링
4. 결측치 처리: 예를 들어 Age의 중앙값 대체, CryoSleep 결측 시 False 등

### 특성 엔지니어링
1. TotalSpend = RoomService + FoodCourt + ShoppingMall + Spa + VRDeck (지출 합계 변수)
2. HasCabin, Deck, NumCabin 등 특징적 파생 피쳐 생성

### 모델 훈련
1. 스태티파이드 K-Fold 또는 간이 validation set 생성
2. XGBoost, LightGBM 각각 훈련 및 검증

### 블렌딩
1. 검증 데이터를 활용해 최적 가중치 탐색 (예: [0.6 * XGB + 0.4 * LGBM])
2. 테스트 데이터에 동일 가중치 적용 후 평균 예측

## 4. 코드 흐름 요약

### 데이터 로딩 & 전처리
```
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# Cabin에서 Deck 추출
df['Deck'] = df['Cabin'].map(lambda x: x.split('/')[0] if pd.notna(x) else 'Unknown')

# 그룹 ID
df['GroupID'] = df['PassengerId'].apply(lambda x: x.split('_')[0])

# 파생 변수
spend_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
df['TotalSpend'] = df[spend_cols].sum(axis=1)
df['LogSpend'] = np.log1p(df['TotalSpend'])
```

### 모델 훈련
```
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

X = df[features]
y = df['Transported'].astype(int)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

xgb = XGBClassifier(...)
lgb = LGBMClassifier(...)
xgb.fit(X_train, y_train)
lgb.fit(X_train, y_train)

pred_xgb = xgb.predict_proba(X_valid)[:,1]
pred_lgb = lgb.predict_proba(X_valid)[:,1]
```

### 블렌딩 (검증 기반 가중 평균)
```
blend_pred = 0.6 * pred_xgb + 0.4 * pred_lgb
# 최적 가중치 계산 (예: grid search)
```

### 테스트 예측 및 제출
```
pred_xgb_test = xgb.predict_proba(test[features])[:,1]
pred_lgb_test = lgb.predict_proba(test[features])[:,1]
final_test_pred = 0.6 * pred_xgb_test + 0.4 * pred_lgb_test
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': final_test_pred > 0.5})
```

## 차별점 (유의미한 포인트)
파생변수 기반 feature engineering이 블렌딩 성능을 뒷받침
예: TotalSpend, Deck 같은 유의미한 값들이 모델 간 보완력을 높임

Soft voting 블렌딩으로 안정성 강한 예측
단일 모델 대비 과적합 방지 및 안정적 성능

가중치 튜닝: validation 기반으로 XGB 중심 또는 LGB 중심 가중치를 유동적으로 조절 가능

## 배울 점 요약
Blending 전략: Soft-voting 블렌딩은 stacking보다 구현이 간단하면서도 실전에서 활용되는 주요 앙상블 방법

Feature Importance 극대화: 파생변수의 퀄리티가 앙상블 효과를 극대화

가중치 최적화 방법: 검증 기반 그리드 검색으로 블렌딩 가중치 조정